In [1]:
import pandas as pd
import numpy as np
from functools import reduce

## Making grand_dict 
w/ keys: filename, delimeter, encoding, filetype, grouping

In [2]:
location = '../Market_Data-Municipality_Level-20210915/'
file_info = '''
life_expectancy.csv ; latin_1 brick
T2D.csv ; latin_1 brick
Sales.csv , utf-8 other
longterm_disease_.csv ; latin_1 brick
andel_blodprop_i_hjertet.csv ; latin_1 brick
patient_data.csv , utf-8 other
attention.csv , utf-8 other
svaert_overvaegt.csv ; latin_1 brick
attention_3.csv ; utf-8 other
svært_overvægt.csv ; latin_1 #
attention_3.xls ; utf-8 #
uddannelse_lang.csv ; latin_1 brick
fravaer.csv ; latin_1 brick
variables_explanations.xlsx ; utf-8 #
fravær.csv ; latin_1 #
who_minimums_fysisk_aktivitet.csv ; latin_1 brick'''.split('\n')[1:]
file_info = [location+i for i in file_info]

In [3]:
grand_dict = {}
for i in file_info:
    contents = i.split(' ')
    #print(contents)
    if contents[3] == '#':
        pass
    else:
        name = contents[0].split('/')[2].split('.')[0]
        grand_dict[name] = {
            'filename' :contents[0],
            'delimeter': contents[1],
            'encoding':contents[2],
            'filetype': contents[0].split('/')[2].split('.')[1],
            'grouping' :contents[3]}
grand_dict['life_expectancy']

{'filename': '../Market_Data-Municipality_Level-20210915/life_expectancy.csv',
 'delimeter': ';',
 'encoding': 'latin_1',
 'filetype': 'csv',
 'grouping': 'brick'}

In [4]:
for i in grand_dict.keys():
    if grand_dict[i]['filetype'] == 'csv':
        grand_dict[i]['dataframe'] = pd.read_csv(grand_dict[i]['filename'], 
                                                 delimiter=grand_dict[i]['delimeter'], 
                                                 encoding=grand_dict[i]['encoding'])

## Dropping and renaming cols

In [5]:
grand_dict['life_expectancy']['dataframe'].rename(columns={'år':'life_expectancy (yrs)'}, inplace=True)

grand_dict['T2D']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['longterm_disease_']['dataframe'].rename(columns={'anddel':'longterm_disease_anddel'}, inplace=True)
grand_dict['longterm_disease_']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['andel_blodprop_i_hjertet']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['patient_data']['dataframe'].drop(columns=['Unnamed: {}'.format(i) for i in range(4,13)],inplace=True)

grand_dict['svaert_overvaegt']['dataframe'].drop(columns=['brick'],inplace=True)
grand_dict['svaert_overvaegt']['dataframe'].rename(columns={'Andel':'svaert_overvaegt Andel'}, inplace=True)

grand_dict['attention_3']['dataframe'].drop(columns=['BRICK_NAME'],inplace=True)
grand_dict['attention_3']['dataframe'].dropna(inplace=True)
grand_dict['attention_3']['dataframe'].rename(columns={'BRICK_NO':'brick_nr'}, inplace=True)
grand_dict['attention_3']['dataframe'] = grand_dict['attention_3']['dataframe'][grand_dict['attention_3']['dataframe']['brick_nr'] != 'Outside DK']
grand_dict['attention_3']['dataframe']['brick_nr'] = grand_dict['attention_3']['dataframe']['brick_nr'].apply(lambda x: int(x))

grand_dict['uddannelse_lang']['dataframe'].drop(columns=['brick', 'brick_navn', 'population'],inplace=True)
grand_dict['uddannelse_lang']['dataframe'].rename(columns={'antal':'uddannelse_lang antal', 'value_per_1000' : 'uddannelse_lang (value_per_1000)'}, inplace=True)

grand_dict['fravaer']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['who_minimums_fysisk_aktivitet']['dataframe'].drop(columns=['brick'],inplace=True)
grand_dict['who_minimums_fysisk_aktivitet']['dataframe'].rename(columns={'Andel':'who_minimums_fysisk_aktivitet Andel'}, inplace=True)

## Combining dfs of equal length

In [6]:
keys = []
for key in grand_dict.keys():
    if grand_dict[key]['grouping'] == 'brick':
        
        keys.append(key)
dfs_brick = [grand_dict[key]['dataframe'] for key in keys]        
df_final = reduce(lambda left,right: pd.merge(left,right,on='brick_nr'), dfs_brick)

In [7]:
df_final.head()

,brick_nr,brick,life_expectancy (yrs),patients,population,patients_per_1000,longterm_disease_anddel,Andel med blodprop i hjertet,svaert_overvaegt Andel,uddannelse_lang antal,uddannelse_lang (value_per_1000),fravær,who_minimums_fysisk_aktivitet Andel
0,101,Bornholm,"77,1905","2248,978472",33046,"68,05599685","0,4065","0,0205","0,1755","6035,258672188186","182,6320484230523","6,2865","0,302"
1,102,Amager,"79,96766666666666","6136,772115",154254,"39,78355255","0,32666666666666666","0,009000000000000001","0,13033333333333333","64810,874711589306","420,15684981646706","5,747333333333334","0,24333333333333332"
2,103,Nørrebro/City,"79,283","3019,148357",88293,"34,19465141","0,305","0,0075","0,1075","44280,90932139913","501,5223100517496","5,499499999999999","0,2215"
3,104,Vesterbro/Christianshavn,"80,05","3387,602593",103052,"32,87274961","0,3075","0,008","0,1","49553,41956356552","480,85839734857666","5,58","0,225"
4,105,Frederiksberg,"79,6455","3574,931136",105260,"33,96286468","0,306","0,008","0,104","52262,04744646349","496,5043458717793","5,5375","0,223"


## What is missing?
Lets print remaining .csv files and the number of unique elements in each column.

In [8]:
for key in grand_dict.keys():
    if grand_dict[key]['grouping'] != 'brick':
        print(key)
        for col in grand_dict[key]['dataframe'].columns:
            print(col, grand_dict[key]['dataframe'][col].nunique())
        print(len(grand_dict[key]['dataframe'][col]),'\n')

Sales
YEAR_MONTH 12
SECTOR_CODE 2
VOLUME 1014
VALUE 72877
WHO_ATC_CODE 47
MUNICIPALITY 99
223673 

patient_data
Hospital 23
Department 428
n_type1 109
n_type2 120
428 

attention
WHO_ATC_5_DERIVED 47
SITE_SHORT 2
PERIOD_YEAR_MONTH_NUM 13
REGION 5
REGIONNAME 5
CITY 656
ZIP 669
CLICKS 659
120843 

attention_3
SITE_SHORT 2
PERIOD_ID 12
COUNT_ 243
WHO_ATC_CODE 31
CATEGORY 5
brick_nr 60
20168 



### attention_3

In [9]:
# we should be able to get attention_3 to fit in the brick group
new_attention_3 = df_final[['brick_nr']].copy()
new_attention_3['fake'] = np.zeros(len(new_attention_3))
df = grand_dict['attention_3']['dataframe'].copy()
for WHO_ATC_CODE in df['WHO_ATC_CODE'].unique():
    for PERIOD_ID in df['PERIOD_ID'].unique():
        for SITE_SHORT in df['SITE_SHORT'].unique():
            temp = df[(df['WHO_ATC_CODE'] == WHO_ATC_CODE) & (df['PERIOD_ID'] == PERIOD_ID) & (df['SITE_SHORT'] == SITE_SHORT)].copy()
            name = WHO_ATC_CODE+'_'+str(PERIOD_ID)+'_'+SITE_SHORT + '_COUNT'
            temp.drop(columns=['WHO_ATC_CODE', 'PERIOD_ID', 'SITE_SHORT', 'CATEGORY'], inplace=True)
            temp.rename(columns={'COUNT_':name}, inplace=True)
            new_attention_3 = new_attention_3.join(temp.set_index('brick_nr'), on='brick_nr')

new_attention_3.drop(columns=['fake'], inplace=True)
new_attention_3.head()

,brick_nr,A10AE04_202006_min_COUNT,A10AE04_202006_pro_COUNT,A10AE04_202005_min_COUNT,A10AE04_202005_pro_COUNT,A10AE04_202004_min_COUNT,A10AE04_202004_pro_COUNT,A10AE04_202003_min_COUNT,A10AE04_202003_pro_COUNT,A10AE04_202002_min_COUNT,...,A10BK04_201911_min_COUNT,A10BK04_201911_pro_COUNT,A10BK04_201910_min_COUNT,A10BK04_201910_pro_COUNT,A10BK04_201909_min_COUNT,A10BK04_201909_pro_COUNT,A10BK04_201908_min_COUNT,A10BK04_201908_pro_COUNT,A10BK04_201907_min_COUNT,A10BK04_201907_pro_COUNT
0,101,NaN,NaN,NaN,1.0,4.0,4.0,4.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102,17.0,21.0,14.0,31.0,19.0,27.0,15.0,26.0,15.0,...,NaN,4.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
2,103,6.0,25.0,9.0,30.0,7.0,34.0,9.0,30.0,6.0,...,NaN,1.0,1.0,4.0,NaN,1.0,NaN,3.0,NaN,3.0
3,104,42.0,62.0,35.0,54.0,38.0,66.0,18.0,64.0,28.0,...,1.0,3.0,3.0,3.0,1.0,1.0,NaN,1.0,3.0,1.0
4,105,4.0,15.0,4.0,9.0,7.0,9.0,6.0,18.0,6.0,...,NaN,3.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dic_min = {}
dic_pro = {}

df = grand_dict['attention_3']['dataframe'].copy()
for i in df.index[:]:
    WHO_ATC_CODE = df['WHO_ATC_CODE'][i]
    brick_nr = df['brick_nr'][i]
    SITE_SHORT = df['SITE_SHORT'][i]
    PERIOD_ID = df['PERIOD_ID'][i]
    
    COUNT_ = df['COUNT_'][i]
    #print('\n',df.iloc[i])
    
    if SITE_SHORT == 'min':
        dic = dic_min
    else:
        dic = dic_pro
    
    if WHO_ATC_CODE in dic.keys():
        if brick_nr in dic[WHO_ATC_CODE].keys():
            dic[WHO_ATC_CODE][brick_nr][PERIOD_ID] = COUNT_
        else:
            dic[WHO_ATC_CODE][brick_nr] = {PERIOD_ID: COUNT_}
    else:
        dic[WHO_ATC_CODE] = {brick_nr : {PERIOD_ID: COUNT_}}
        
    if SITE_SHORT == 'min':
        dic_min = dic
    else:
        dic_pro = dic
        
        
dic_min['A10AE04'][502]

{202006: 13,
 202005: 22,
 202004: 7,
 202003: 12,
 202002: 17,
 202001: 21,
 201912: 23,
 201911: 79,
 201910: 34,
 201909: 23,
 201908: 27,
 201907: 6}

In [11]:
dict_of_df = {k: pd.DataFrame(v) for k,v in dic_min.items()}
df_attention_3_min = pd.concat(dict_of_df, axis=1).T

dict_of_df = {k: pd.DataFrame(v) for k,v in dic_pro.items()}
df_attention_3_pro = pd.concat(dict_of_df, axis=1).T

df_attention_3_min # /min

201907  201908  201909  201910  201911  201912  202001  202002  \
A10AE04 308     3.0     NaN     3.0     3.0     2.0     3.0     1.0     1.0   
        502     6.0    27.0    23.0    34.0    79.0    23.0    21.0    17.0   
        102     7.0     6.0     7.0    12.0    12.0    11.0     8.0    15.0   
        303     3.0     1.0     NaN     6.0     4.0     2.0     6.0     2.0   
        115     8.0    10.0    13.0     4.0    15.0     6.0     9.0    20.0   
...             ...     ...     ...     ...     ...     ...     ...     ...   
A10BK04 410     NaN     1.0     NaN     NaN     NaN     NaN     NaN     NaN   
        502     2.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
        313     1.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
        307     3.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
        403     5.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

             202003  202004  202005  202006  
A10AE04 308     1.0     4.0     5.0     1.0  
        502    12.0     7.0    22.0    13.0  
        102    15.0    19.0    14.0    17.0  
        303     1.0     2.0     4.0     1.0  
        115    10.0    17.0     4.0     4.0  
...             ...     ...     ...     ...  
A10BK04 410     NaN     NaN     NaN     NaN  
        502     NaN     NaN     NaN     NaN  
        313     NaN     NaN     NaN     NaN  
        307     NaN     NaN     NaN     NaN  
        403     NaN     NaN     NaN     NaN  

[1247 rows x 12 columns]

### Sales

In [24]:
# Now for sales

# First lets grab a copy
df = grand_dict['Sales']['dataframe'].copy()
df = df[df['SECTOR_CODE']=='P']
df.drop(columns=['SECTOR_CODE'], inplace=True)
value_dict = {}
volume_dict = {}

for i in df.index:
    MUNICIPALITY, WHO_ATC_CODE, YEAR_MONTH = df['MUNICIPALITY'][i], df['WHO_ATC_CODE'][i], df['YEAR_MONTH'][i]
    VOLUME, VALUE = df['VOLUME'][i], df['VALUE'][i]
    
    if WHO_ATC_CODE in value_dict.keys():
        if MUNICIPALITY in value_dict[WHO_ATC_CODE].keys():
            value_dict[WHO_ATC_CODE][MUNICIPALITY][YEAR_MONTH] =VALUE
            volume_dict[WHO_ATC_CODE][MUNICIPALITY][YEAR_MONTH] =VOLUME   
        else:
            value_dict[WHO_ATC_CODE][MUNICIPALITY] = {YEAR_MONTH :VALUE}
            volume_dict[WHO_ATC_CODE][MUNICIPALITY] = {YEAR_MONTH :VOLUME}
    else:
        for dic, x in zip([value_dict, volume_dict],[VALUE, VOLUME]):
            dic[WHO_ATC_CODE] = {}
            dic[WHO_ATC_CODE][MUNICIPALITY]= {YEAR_MONTH: x}
            
            
dict_of_df = {k: pd.DataFrame(v) for k,v in value_dict.items()}
df_value = pd.concat(dict_of_df, axis=1)

dict_of_df = {k: pd.DataFrame(v) for k,v in volume_dict.items()}
df_volume = pd.concat(dict_of_df, axis=1)

df_value.T.head(3)
df_volume.T#.head(3)

202004  202005  202006  202007  202008  202009  \
A10BD20 Allerød            3.0    -1.0     2.0     3.0     2.0     2.0   
        Randers            3.0    -1.0    -2.0     1.0     2.0    -2.0   
        Frederiksberg      4.0     2.0     1.0     1.0     1.0     5.0   
        Odense            13.0     5.0    -2.0    18.0     1.0     9.0   
        Lyngby-Taarbæk     9.0     1.0     1.0    10.0     2.0     7.0   
...                        ...     ...     ...     ...     ...     ...   
A10AD04 Fredensborg        NaN     NaN     1.0     NaN     NaN     NaN   
        Aabenraa           NaN     1.0     NaN     NaN     NaN     NaN   
        Fredericia         NaN     NaN     NaN     NaN     NaN     NaN   
        Kolding            NaN     NaN     NaN     NaN     NaN    -1.0   
        Rebild             NaN     NaN     NaN     NaN     NaN    -2.0   

                        202010  202011  202012  202101  202102  202103  
A10BD20 Allerød            2.0     1.0     1.0     5.0     1.0     1.0  
        Randers           -1.0    -1.0     5.0    -1.0     3.0     5.0  
        Frederiksberg      5.0    -1.0    -1.0     4.0     0.0     5.0  
        Odense            -1.0    13.0    27.0    -2.0     2.0    15.0  
        Lyngby-Taarbæk     0.0     1.0     2.0    -1.0     4.0    -3.0  
...                        ...     ...     ...     ...     ...     ...  
A10AD04 Fredensborg        NaN     NaN     NaN     NaN     NaN     3.0  
        Aabenraa           NaN     NaN     NaN     NaN     2.0     NaN  
        Fredericia         NaN     1.0     NaN     NaN     NaN     1.0  
        Kolding            NaN     NaN     NaN     NaN     NaN     NaN  
        Rebild             NaN     NaN     NaN     NaN     NaN     NaN  

[3582 rows x 12 columns]

### patient_data

In [15]:
df = grand_dict['patient_data']['dataframe'].copy()
df_patient_data = pd.DataFrame(df[['n_type1','n_type2']].values, 
                   index=[
                       df['Hospital'].values,
                       df['Department'].values], 
                   columns='n_type1, n_type2'.split(', '))
df_patient_data.head()

n_type1 n_type2
Aalborg Universitetshospital Alb Akut Modtageafdeling       -       -
                             Alb Børne Akut Amb             -       -
                             Alb Børneafd. amb.           683       -
                             Alb Børneafdeling              -       -
                             Alb Dagafdeling Børn          <5       -

### attention

In [16]:
df = grand_dict['attention']['dataframe'].copy()
df['Region'] = df['REGIONNAME']+' '+ df['REGION'].astype(str)
df['City'] = df['ZIP'].astype(str)+' '+ df['CITY']
df.drop(columns='CITY, REGION, REGIONNAME, ZIP'.split(', '), inplace=True)

#df.head()

In [17]:
clicks_dict_pro = {} # ZIP = {}, 
clicks_dict_min = {}

for i in df.index[:]:
    Region,City , WHO_ATC_CODE, YEAR_MONTH = df['Region'][i], df['City'][i], df['WHO_ATC_5_DERIVED'][i], df['PERIOD_YEAR_MONTH_NUM'][i]
    clicks, SITE_SHORT = df['CLICKS'][i], df['SITE_SHORT'][i]
    #print(Region)
    if SITE_SHORT == 'pro':
        if Region in clicks_dict_pro.keys():
            if City in clicks_dict_pro[Region].keys():
                if WHO_ATC_CODE in clicks_dict_pro[Region][City].keys():
                    clicks_dict_pro[Region][City][WHO_ATC_CODE][YEAR_MONTH] = clicks
                else:
                    clicks_dict_pro[Region][City][WHO_ATC_CODE] = {YEAR_MONTH: clicks}
            else:
                clicks_dict_pro[Region][City] = {WHO_ATC_CODE :{YEAR_MONTH :clicks}}
        else:
            clicks_dict_pro[Region] = {}
            clicks_dict_pro[Region][City]= {WHO_ATC_CODE: {YEAR_MONTH : clicks}}
    else:
        if Region in clicks_dict_min.keys():
            if City in clicks_dict_min[Region].keys():
                if WHO_ATC_CODE in clicks_dict_min[Region][City].keys():
                    clicks_dict_min[Region][City][WHO_ATC_CODE][YEAR_MONTH] = clicks
                else:
                    clicks_dict_min[Region][City][WHO_ATC_CODE] = {YEAR_MONTH: clicks}
            else:
                clicks_dict_min[Region][City] = {WHO_ATC_CODE :{YEAR_MONTH :clicks}}
        else:
            clicks_dict_min[Region] = {}
            clicks_dict_min[Region][City]= {WHO_ATC_CODE: {YEAR_MONTH : clicks}}
#clicks_dict_pro
            


In [18]:
reform = {(level1_key, level2_key, level3_key): values
   ...:           for level1_key, level2_dict in clicks_dict_min.items()
   ...:           for level2_key, level3_dict in level2_dict.items()
   ...:           for level3_key, values      in level3_dict.items()}
df_attention_clicks_min = pd.DataFrame(reform).T

reform = {(level1_key, level2_key, level3_key): values
   ...:           for level1_key, level2_dict in clicks_dict_pro.items()
   ...:           for level2_key, level3_dict in level2_dict.items()
   ...:           for level3_key, values      in level3_dict.items()}
df_attention_clicks_pro = pd.DataFrame(reform).T

## Now lets save all the files

In [22]:
df_value.T.to_csv('../Cleaned_Market_data_anton/df_value.csv')
df_volume.T.to_csv('../Cleaned_Market_data_anton/df_volume.csv')

In [19]:
loc = '../antonsdata/'

# bricks
df_final.to_csv(loc+'df_bricks.csv')

# Attention 3
df_attention_3_min.to_csv(loc+'new_attention_3_min.csv')
df_attention_3_pro.to_csv(loc+'new_attention_3_pro.csv')

# Sales
df_value.T.to_csv(loc+'df_value.csv')
df_volume.T.to_csv(loc+'df_volume.csv')

# Patient data
df_patient_data.to_csv(loc+'df_patient_data.csv')

# attention
df_attention_clicks_min.to_csv(loc+'df_attention_clicks_min.csv')
df_attention_clicks_pro.to_csv(loc+'df_attention_clicks_pro.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../antonsdata/df_bricks.csv'

## And try loading them again

In [ ]:
# bricks
dfA = pd.read_csv(loc+'df_bricks.csv',index_col=[0],decimal= ',')

# Attention 3
dfB1 = pd.read_csv(loc+'new_attention_3_pro.csv',index_col=[0,1])
dfB2 = pd.read_csv(loc+'new_attention_3_min.csv',index_col=[0,1])

# Sales
dfC1 = pd.read_csv(loc+'df_value.csv',index_col=[0,1])
dfC2 = pd.read_csv(loc+'df_volume.csv',index_col=[0,1])

# Patient data
dfD = pd.read_csv(loc+'df_patient_data.csv',index_col=[0,1])

# attention

dfE1 = pd.read_csv(loc+'df_attention_clicks_min.csv',index_col=[0,1,2])
dfE2 = pd.read_csv(loc+'df_attention_clicks_pro.csv',index_col=[0,1,2])



dfC1

In [ ]:
grand_dict['Sales']['dataframe']